## Weekly Twitter, Facebook and Instagram Organic Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from time import strptime

### Twitter
Read in data export from twitter platform, wrangle, and export cleaned csv

In [2]:
# 3/7 - 3/24
twitter = pd.read_csv('twitter_weekly_pull.csv')
print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 40 columns):
Tweet id                        25 non-null float64
Tweet permalink                 25 non-null object
Tweet text                      25 non-null object
time                            25 non-null object
impressions                     25 non-null int64
engagements                     25 non-null int64
engagement rate                 25 non-null float64
retweets                        25 non-null int64
replies                         25 non-null int64
likes                           25 non-null int64
user profile clicks             25 non-null int64
url clicks                      25 non-null int64
hashtag clicks                  25 non-null int64
detail expands                  25 non-null int64
permalink clicks                25 non-null int64
app opens                       25 non-null int64
app installs                    25 non-null int64
follows                         25 non-n

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1.109850e+18,https://twitter.com/KUWTK/status/1109854888635...,There’s no chill in this family 😂😂😂 #KUWTK htt...,2019-03-24 16:30 +0000,53787,8068,0.149999,109,17,641,...,-,-,-,-,-,-,-,-,-,-
1,1.109490e+18,https://twitter.com/KUWTK/status/1109492501776...,"Some of you have never lost a $75,000 diamond ...",2019-03-23 16:30 +0000,85906,2334,0.027169,257,36,1179,...,-,-,-,-,-,-,-,-,-,-
2,1.109130e+18,https://twitter.com/KUWTK/status/1109130114623...,The new season of #KUWTK 🤣 &gt; https://t.co/g...,2019-03-22 16:30 +0000,80813,14077,0.174192,226,22,1169,...,-,-,-,-,-,-,-,-,-,-
3,1.108770e+18,https://twitter.com/KUWTK/status/1108767735289...,Are you the real Meme Queen? Reply with your b...,2019-03-21 16:30 +0000,26404,1266,0.047947,19,19,214,...,-,-,-,-,-,-,-,-,-,-
4,1.108040e+18,https://twitter.com/KUWTK/status/1108041887674...,#TBT that time Scott should’ve brought an umbr...,2019-03-19 16:25 +0000,45832,5615,0.122513,90,32,631,...,-,-,-,-,-,-,-,-,-,-


In [3]:
# filter out unneeded columns
twitter = twitter.filter(['Tweet permalink', 'Tweet text', 'time', 'impressions', 'engagements',
                          'engagement rate', 'retweets', 'replies', 'likes', 'media views', 'media engagements'])
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 11 columns):
Tweet permalink      25 non-null object
Tweet text           25 non-null object
time                 25 non-null object
impressions          25 non-null int64
engagements          25 non-null int64
engagement rate      25 non-null float64
retweets             25 non-null int64
replies              25 non-null int64
likes                25 non-null int64
media views          25 non-null int64
media engagements    25 non-null int64
dtypes: float64(1), int64(7), object(3)
memory usage: 2.2+ KB


In [4]:
#create 'post type' column indicating whether reply or original post
twitter['post type'] = twitter['Tweet text'].map(lambda x: "reply" if x[0] == '@' else "post")

# create 'vis eng' column calculating visible engagements by adding retweets, replies, likes
twitter['vis eng'] = twitter['retweets'] + twitter['replies'] + twitter['likes']

# create 'view Rate' and 'completion rate' columns by dividing media views and engagements by impressions, respectively
twitter['view rate'] = twitter['media views']/twitter['impressions']
twitter['completion rate'] = twitter['media engagements']/twitter['media views']
twitter['completion rate'] = twitter['completion rate'].fillna(0)

# add 'episode' column assigning episode number: first 2 digits = season number, last 2 digits = episode number
twitter['episode'] = 'none'

# convert 'date' column to datetime
twitter['time'] = pd.to_datetime(twitter['time'])

# create 'in-show' column - show aired on 5/5 at 9pm ET, so tweets between GMT 5/5 7PM - 5/6 5AM are general in-show
## 'West Coast in-show' - 5/5 9-10pm PT so tweets between GMT (+7) 5/6 4-5am
## 'East Coast in-show' - 5/5 6-10pm ET so tweets between GMT (+4) 5/6 12am - 2am
twitter['in-show'] = twitter['time'].map(lambda x: "West Coast in-show" if x >= pd.to_datetime(['2019-05-06 04:00:00']) and x <= pd.to_datetime(['2019-05-06 05:00:00']) else
                                         "East Coast in-show" if x >= pd.to_datetime(['2019-05-06 00:00:00']) and x < pd.to_datetime(['2019-05-06 04:00:00']) else "not")

#Phase = premiere/continuity/finale
twitter['phase'] = twitter['time'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-03-25 00:00:00']) and x <= pd.to_datetime(['2019-04-03 11:59:00']) else 
                                       "continuity")

#messaging = tune-in/awareness/catch-up
twitter['messaging'] = twitter['Tweet text'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

# create empty 'Votes' column
twitter['votes'] = np.nan

twitter['content series'] = np.nan

twitter['asset type'] = np.nan

twitter['subject'] = np.nan

print(twitter.info())
twitter

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 23 columns):
Tweet permalink      25 non-null object
Tweet text           25 non-null object
time                 25 non-null datetime64[ns]
impressions          25 non-null int64
engagements          25 non-null int64
engagement rate      25 non-null float64
retweets             25 non-null int64
replies              25 non-null int64
likes                25 non-null int64
media views          25 non-null int64
media engagements    25 non-null int64
post type            25 non-null object
vis eng              25 non-null int64
view rate            25 non-null float64
completion rate      25 non-null float64
episode              25 non-null object
in-show              25 non-null object
phase                25 non-null object
messaging            25 non-null object
votes                0 non-null float64
content series       0 non-null float64
asset type           0 non-null float64
subject       

,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,media views,...,view rate,completion rate,episode,in-show,phase,messaging,votes,content series,asset type,subject
0,https://twitter.com/KUWTK/status/1109854888635...,There’s no chill in this family 😂😂😂 #KUWTK htt...,2019-03-24 16:30:00,53787,8068,0.149999,109,17,641,16451,...,0.305855,0.412133,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
1,https://twitter.com/KUWTK/status/1109492501776...,"Some of you have never lost a $75,000 diamond ...",2019-03-23 16:30:00,85906,2334,0.027169,257,36,1179,0,...,0.000000,0.000000,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
2,https://twitter.com/KUWTK/status/1109130114623...,The new season of #KUWTK 🤣 &gt; https://t.co/g...,2019-03-22 16:30:00,80813,14077,0.174192,226,22,1169,24407,...,0.302018,0.485762,none,not,continuity,tune-in,NaN,NaN,NaN,NaN
3,https://twitter.com/KUWTK/status/1108767735289...,Are you the real Meme Queen? Reply with your b...,2019-03-21 16:30:00,26404,1266,0.047947,19,19,214,4594,...,0.173989,0.152373,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
4,https://twitter.com/KUWTK/status/1108041887674...,#TBT that time Scott should’ve brought an umbr...,2019-03-19 16:25:00,45832,5615,0.122513,90,32,631,9985,...,0.217861,0.433751,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
5,https://twitter.com/KUWTK/status/1107676146282...,Like for @kourtneykardash 😂 RT for @krisjenner...,2019-03-18 16:12:00,62816,7579,0.120654,147,47,855,14883,...,0.236930,0.399987,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
6,https://twitter.com/KUWTK/status/1107675726654...,@Nick86782529 You're welcome 😘,2019-03-18 16:10:00,815,13,0.015951,1,0,0,0,...,0.000000,0.000000,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
7,https://twitter.com/KUWTK/status/1107675608920...,@ambermarie4522 @KimKardashian 😂😂😂,2019-03-18 16:10:00,601,10,0.016639,1,0,2,0,...,0.000000,0.000000,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
8,https://twitter.com/KUWTK/status/1107310630506...,Be sure to wish @robkardashian a very happy bi...,2019-03-17 16:00:00,372358,17832,0.047889,97,58,2454,89547,...,0.240486,0.153551,none,not,continuity,Awareness,NaN,NaN,NaN,NaN
9,https://twitter.com/KUWTK/status/1106948237884...,Nobody:\nKim: You’re the least interesting to ...,2019-03-16 16:00:00,37766,1024,0.027114,105,8,594,0,...,0.000000,0.000000,none,not,continuity,Awareness,NaN,NaN,NaN,NaN


In [5]:
#standardize column names i.e. make all lowercase
twitter.columns = map(str.lower, twitter.columns)
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 23 columns):
tweet permalink      25 non-null object
tweet text           25 non-null object
time                 25 non-null datetime64[ns]
impressions          25 non-null int64
engagements          25 non-null int64
engagement rate      25 non-null float64
retweets             25 non-null int64
replies              25 non-null int64
likes                25 non-null int64
media views          25 non-null int64
media engagements    25 non-null int64
post type            25 non-null object
vis eng              25 non-null int64
view rate            25 non-null float64
completion rate      25 non-null float64
episode              25 non-null object
in-show              25 non-null object
phase                25 non-null object
messaging            25 non-null object
votes                0 non-null float64
content series       0 non-null float64
asset type           0 non-null float64
subject       

In [6]:
#reorder columns
twitter = twitter[['post type','time', 'tweet permalink', 'tweet text', 'episode',  'in-show', 'phase', 'messaging', 'content series', 'asset type', 'subject',
                   'impressions', 'engagements', 'engagement rate', 'retweets', 'replies', 'likes', 'votes', 'vis eng',
                   'media views', 'media engagements', 'view rate', 'completion rate']]
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 23 columns):
post type            25 non-null object
time                 25 non-null datetime64[ns]
tweet permalink      25 non-null object
tweet text           25 non-null object
episode              25 non-null object
in-show              25 non-null object
phase                25 non-null object
messaging            25 non-null object
content series       0 non-null float64
asset type           0 non-null float64
subject              0 non-null float64
impressions          25 non-null int64
engagements          25 non-null int64
engagement rate      25 non-null float64
retweets             25 non-null int64
replies              25 non-null int64
likes                25 non-null int64
votes                0 non-null float64
vis eng              25 non-null int64
media views          25 non-null int64
media engagements    25 non-null int64
view rate            25 non-null float64
completion rate

In [7]:
# convert new dataframe to csv "twitter_weekly.csv"
twitter.to_csv('twitter_weekly.csv')

### Nielsen
Read in data export from Nielsen UI, wrangle and write to csv.

In [8]:
# read in file for current week's episode
# air time: 5/5 - 9pm ET
nielsen = pd.read_csv('nielsen_weekly_pull.csv')
nielsen.info()
nielsen.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8206 entries, 0 to 8205
Data columns (total 14 columns):
Social Content Ratings    8198 non-null object
Unnamed: 1                8196 non-null object
Unnamed: 2                8187 non-null object
Unnamed: 3                8187 non-null object
Unnamed: 4                8187 non-null object
Unnamed: 5                8183 non-null object
Unnamed: 6                8183 non-null object
Unnamed: 7                8181 non-null object
Unnamed: 8                8181 non-null object
Unnamed: 9                8181 non-null object
Unnamed: 10               8181 non-null object
Unnamed: 11               8181 non-null object
Unnamed: 12               8181 non-null object
Unnamed: 13               7012 non-null object
dtypes: object(14)
memory usage: 897.6+ KB


,Social Content Ratings,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,http://scr.nielsensocial.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Copyright 2019 The Nielsen Company. All right...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Network,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Program,Keeping Up With The Kardashians,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Program Genre Type,Comedy Series; Reality Series; Docs & Special ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Season Number,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Episode,Legally Brunette,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Episode Number,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EPG Program ID,SH009675940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# toss row indices 0-9 into a new dataframe and isolate episode name
episode_details = nielsen.iloc[:13]
#season = episode_details.iat[6,1]
episode_details['episode'] = episode_details.iat[6,1] + '0' + episode_details.iat[8,1]
episode_details['name'] = episode_details.iat[7,1]
episode_details['air date'] = episode_details.iat[11,1]
episode_details['air time'] = episode_details.iat[12,1]
episode_details.drop(episode_details.index[1:],inplace=True)
episode_details = episode_details.loc[:, ['episode','name','air date','air time']]
episode_details

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,episode,name,air date,air time
0,1605,Legally Brunette,5/5/19,09:00 PM EDT


In [10]:
# bring up row index 17 as header and drop row indices 0-16
nielsen.drop(nielsen.index[:17], inplace=True)
nielsen.columns = nielsen.iloc[0]
nielsen = nielsen[1:]
nielsen = nielsen.reset_index(drop = True)
nielsen.head(15)

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan
0,Twitter,568,1169,45023,91821,45355,92990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Retweets,3269,5069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Replies,1029,1841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Quotes,91,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Likes,43225,84810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,User,User Profile Url,User ID,Followers,Time Sent (Local),Local Date,Time Sent (GMT),Tweet ID,Tweet,Tweet URL,Interaction Type,Tweet Type,Root ID,Root URL
8,ChisomAdaora,https://twitter.com/ChisomAdaora,8.46719E+17,339,06:50:46 PM EDT,5/5/19,2019-05-05T22:50:46Z,1.12517E+18,RT @KrisJenner: Happy #KUWTKSunday! Brand new ...,https://twitter.com/statuses/1125171001942261765,engagement,retweet,1.12504E+18,https://twitter.com/statuses/1125042685373345792
9,imabossluna,https://twitter.com/imabossluna,257193831,262,07:44:22 PM EDT,5/5/19,2019-05-05T23:44:22Z,1.12518E+18,RT @KrisJenner: Happy #KUWTKSunday! Brand new ...,https://twitter.com/statuses/1125184493357285376,engagement,retweet,1.12504E+18,https://twitter.com/statuses/1125042685373345792


In [11]:
# calculate value for total mentions and add as column
mentions = nielsen.iloc[8:]
nielsen['total mentions'] = mentions['Unique Authors'].count()

nielsen.head()

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan,total mentions
0,Twitter,568,1169,45023,91821,45355,92990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8180
1,NaN,NaN,Retweets,3269,5069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8180
2,NaN,NaN,Replies,1029,1841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8180
3,NaN,NaN,Quotes,91,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8180
4,NaN,NaN,Likes,43225,84810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8180


In [12]:
# keep only relevant columns and drop all rows except first
nielsen = nielsen[['Unique Authors','Original Content','Unique Engagers','Interactions','total mentions']]
nielsen.drop(nielsen.index[1:], inplace=True)
nielsen.head()

17,Unique Authors,Original Content,Unique Engagers,Interactions,total mentions
0,568,1169,45023,92990,8180


In [13]:
# rename and reorder columns
nielsen.columns = ['unique users','total social convos','unique authors/engagers','interactions','total mentions']
nielsen['episode'], nielsen['name'], nielsen['air date'], nielsen['air time'] = episode_details['episode'], episode_details['name'], episode_details['air date'], episode_details['air time']
nielsen = nielsen[['episode','name','air date','air time','unique users','total social convos','unique authors/engagers','interactions','total mentions']]
nielsen.head()

,episode,name,air date,air time,unique users,total social convos,unique authors/engagers,interactions,total mentions
0,1605,Legally Brunette,5/5/19,09:00 PM EDT,568,1169,45023,92990,8180


In [14]:
nielsen.to_csv('nielsen_weekly.csv')

### Facebook

Read in data export from Facebook platform, wrangle, and write to csv.

In [8]:
fb = pd.read_csv('fb_weekly_pull.csv')
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 56 columns):
Post ID                                                                        17 non-null object
Permalink                                                                      17 non-null object
Post Message                                                                   17 non-null object
Type                                                                           17 non-null object
Countries                                                                      0 non-null float64
Languages                                                                      0 non-null float64
Posted                                                                         17 non-null object
Audience Targeting                                                             17 non-null object
Lifetime Post Total Reach                                                      18 non-null object
Lifetime Post o

,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,Lifetime Post Total Reach,Lifetime Post organic reach,...,Lifetime Matched Audience Targeting Consumptions by Type - link clicks,Lifetime Matched Audience Targeting Consumptions by Type - video play,Lifetime Negative Feedback from Users by Type - hide_all_clicks,Lifetime Negative Feedback from Users by Type - hide_clicks,Lifetime Negative Feedback from Users by Type - report_spam_clicks,Lifetime Negative Feedback from Users by Type - unlike_page_clicks,Lifetime Negative Feedback by Type - hide_all_clicks,Lifetime Negative Feedback by Type - hide_clicks,Lifetime Negative Feedback by Type - report_spam_clicks,Lifetime Negative Feedback by Type - unlike_page_clicks
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lifetime: The number of people who had your Pa...,Lifetime: The number of people who had your Pa...,...,NaN,NaN,Lifetime: The number of times people have give...,NaN,NaN,NaN,Lifetime: The number of people who have given ...,NaN,NaN,NaN
1,166720031891_10156075839731892,https://www.facebook.com/kuwtk/posts/101560758...,"Some of you have never lost a $75,000 diamond ...",Status,NaN,NaN,3/23/19 9:25,,1215980,1215980,...,3.0,NaN,448,76.0,NaN,NaN,435,67.0,NaN,NaN
2,166720031891_10156072364381892,https://www.facebook.com/kuwtk/posts/101560723...,"These queens werk hard, but they play way hard...",Video,NaN,NaN,3/22/19 9:30,,2688298,658986,...,16.0,215189.0,49,39.0,1.0,NaN,50,34.0,1.0,NaN
3,166720031891_10156070084321892,https://www.facebook.com/kuwtk/posts/101560700...,Mood,Photo,NaN,NaN,3/20/19 12:27,,910196,910196,...,1.0,NaN,78,54.0,NaN,1.0,77,54.0,NaN,1.0
4,166720031891_10156061041786892,https://www.facebook.com/kuwtk/posts/101560610...,Leave a ☘ to wish Bob a happy birthday!,Video,NaN,NaN,3/17/19 9:00,,535502,535502,...,NaN,16178.0,43,31.0,NaN,NaN,43,31.0,NaN,NaN


In [9]:
#drop unwanted rows and columns
fb.drop(fb.index[0], inplace=True)
fb = fb.filter(['Permalink','Posted','Post Message','Type','Lifetime Post organic reach','Lifetime Post Organic Impressions','Lifetime Engaged Users','Lifetime Organic Video Views.1','Lifetime Organic views to 95%.1','Lifetime Talking About This (Post) by action type - like','Lifetime Talking About This (Post) by action type - comment','Lifetime Talking About This (Post) by action type - share'])
fb = fb.reset_index(drop=True)
fb.head()

,Permalink,Posted,Post Message,Type,Lifetime Post organic reach,Lifetime Post Organic Impressions,Lifetime Engaged Users,Lifetime Organic Video Views.1,Lifetime Organic views to 95%.1,Lifetime Talking About This (Post) by action type - like,Lifetime Talking About This (Post) by action type - comment,Lifetime Talking About This (Post) by action type - share
0,https://www.facebook.com/kuwtk/posts/101560758...,3/23/19 9:25,"Some of you have never lost a $75,000 diamond ...",Status,1215980,1843895,105186,0,0,30761.0,5290.0,2772
1,https://www.facebook.com/kuwtk/posts/101560723...,3/22/19 9:30,"These queens werk hard, but they play way hard...",Video,658986,895269,228019,432311,42212,6512.0,743.0,603
2,https://www.facebook.com/kuwtk/posts/101560700...,3/20/19 12:27,Mood,Photo,910196,1245215,41842,0,0,20305.0,3913.0,2929
3,https://www.facebook.com/kuwtk/posts/101560610...,3/17/19 9:00,Leave a ☘ to wish Bob a happy birthday!,Video,535502,712341,45423,284921,133726,3824.0,489.0,114
4,https://www.facebook.com/kuwtk/posts/101560610...,3/16/19 9:00,"11 years ago on this day, Kris wanted her very...",Video,982649,1313058,99623,561235,51084,5790.0,2354.0,456


In [10]:
#rename columns
fb.columns = ['link','date posted','message','media type','reach','organic impressions','engagement','organic video views','organic video completes','likes','comments','shares']
fb.head()

,link,date posted,message,media type,reach,organic impressions,engagement,organic video views,organic video completes,likes,comments,shares
0,https://www.facebook.com/kuwtk/posts/101560758...,3/23/19 9:25,"Some of you have never lost a $75,000 diamond ...",Status,1215980,1843895,105186,0,0,30761.0,5290.0,2772
1,https://www.facebook.com/kuwtk/posts/101560723...,3/22/19 9:30,"These queens werk hard, but they play way hard...",Video,658986,895269,228019,432311,42212,6512.0,743.0,603
2,https://www.facebook.com/kuwtk/posts/101560700...,3/20/19 12:27,Mood,Photo,910196,1245215,41842,0,0,20305.0,3913.0,2929
3,https://www.facebook.com/kuwtk/posts/101560610...,3/17/19 9:00,Leave a ☘ to wish Bob a happy birthday!,Video,535502,712341,45423,284921,133726,3824.0,489.0,114
4,https://www.facebook.com/kuwtk/posts/101560610...,3/16/19 9:00,"11 years ago on this day, Kris wanted her very...",Video,982649,1313058,99623,561235,51084,5790.0,2354.0,456


In [11]:
#fb = fb.fillna(0)

cols = ['likes','comments','shares','reach','organic impressions','engagement','organic video views','organic video completes']

for col in cols:
    fb[col] = fb[col].fillna(0)
    fb[col] = fb[col].astype(int)
    
fb.info()
fb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 12 columns):
link                       17 non-null object
date posted                17 non-null object
message                    17 non-null object
media type                 17 non-null object
reach                      17 non-null int64
organic impressions        17 non-null int64
engagement                 17 non-null int64
organic video views        17 non-null int64
organic video completes    17 non-null int64
likes                      17 non-null int64
comments                   17 non-null int64
shares                     17 non-null int64
dtypes: int64(8), object(4)
memory usage: 1.7+ KB


,link,date posted,message,media type,reach,organic impressions,engagement,organic video views,organic video completes,likes,comments,shares
0,https://www.facebook.com/kuwtk/posts/101560758...,3/23/19 9:25,"Some of you have never lost a $75,000 diamond ...",Status,1215980,1843895,105186,0,0,30761,5290,2772
1,https://www.facebook.com/kuwtk/posts/101560723...,3/22/19 9:30,"These queens werk hard, but they play way hard...",Video,658986,895269,228019,432311,42212,6512,743,603
2,https://www.facebook.com/kuwtk/posts/101560700...,3/20/19 12:27,Mood,Photo,910196,1245215,41842,0,0,20305,3913,2929
3,https://www.facebook.com/kuwtk/posts/101560610...,3/17/19 9:00,Leave a ☘ to wish Bob a happy birthday!,Video,535502,712341,45423,284921,133726,3824,489,114
4,https://www.facebook.com/kuwtk/posts/101560610...,3/16/19 9:00,"11 years ago on this day, Kris wanted her very...",Video,982649,1313058,99623,561235,51084,5790,2354,456
5,https://www.facebook.com/kuwtk/posts/101560601...,3/15/19 9:10,Share to bless someone’s feed with Kris Jenner 💁,Video,193144,263125,18543,130964,25519,1148,225,126
6,https://www.facebook.com/kuwtk/posts/101560588...,3/14/19 16:28,"What do you prefer, fam: nude lip or red? Watc...",Video,362380,495622,57502,282608,134073,2044,101,77
7,https://www.facebook.com/kuwtk/posts/101560582...,3/14/19 10:30,"Relationships are a gamble, but this family wi...",Video,683772,904997,210236,333180,107647,8289,1610,1093
8,https://www.facebook.com/kuwtk/posts/101560581...,3/14/19 9:00,"2 seasons ago, Koko’s journey through motherho...",Video,397401,537577,49583,255771,82630,2967,47,100
9,https://www.facebook.com/kuwtk/posts/101560552...,3/12/19 9:30,"October 14th, 2007 KUWTK was born.",Video,620461,819944,70448,360280,192568,1402,218,81


In [12]:
# change date column to date data type
fb['date posted'] = pd.to_datetime(fb['date posted'])

# add calculated columns
fb['episode'] = 'none'
fb['answers'] = np.nan
fb['vis eng'] = fb['likes'] + fb['comments'] + fb['shares']
fb['engagement rate'] = fb['engagement']/fb['reach']
fb['view rate'] = fb['organic video views']/fb['organic impressions']
fb['completion rate'] = fb['organic video completes']/fb['organic video views']
fb['completion rate'] = fb['completion rate'].fillna(0.0)
fb['content series'] = np.nan
fb['asset type'] = np.nan
fb['subject'] = np.nan

#Phase = pre-premiere/premiere/continuity/finale
fb['phase'] = fb['date posted'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-03-25 00:00:00']) and x <= pd.to_datetime(['2019-04-03 11:59:00']) else "continuity")

#messaging = tune-in/awareness/catch-up
fb['message'] = fb['message'].replace(np.nan, 'none')

fb['messaging'] = fb['message'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

fb.head()

,link,date posted,message,media type,reach,organic impressions,engagement,organic video views,organic video completes,likes,...,answers,vis eng,engagement rate,view rate,completion rate,content series,asset type,subject,phase,messaging
0,https://www.facebook.com/kuwtk/posts/101560758...,2019-03-23 09:25:00,"Some of you have never lost a $75,000 diamond ...",Status,1215980,1843895,105186,0,0,30761,...,NaN,38823,0.086503,0.000000,0.000000,NaN,NaN,NaN,continuity,Awareness
1,https://www.facebook.com/kuwtk/posts/101560723...,2019-03-22 09:30:00,"These queens werk hard, but they play way hard...",Video,658986,895269,228019,432311,42212,6512,...,NaN,7858,0.346015,0.482884,0.097643,NaN,NaN,NaN,continuity,tune-in
2,https://www.facebook.com/kuwtk/posts/101560700...,2019-03-20 12:27:00,Mood,Photo,910196,1245215,41842,0,0,20305,...,NaN,27147,0.045970,0.000000,0.000000,NaN,NaN,NaN,continuity,Awareness
3,https://www.facebook.com/kuwtk/posts/101560610...,2019-03-17 09:00:00,Leave a ☘ to wish Bob a happy birthday!,Video,535502,712341,45423,284921,133726,3824,...,NaN,4427,0.084823,0.399978,0.469344,NaN,NaN,NaN,continuity,Awareness
4,https://www.facebook.com/kuwtk/posts/101560610...,2019-03-16 09:00:00,"11 years ago on this day, Kris wanted her very...",Video,982649,1313058,99623,561235,51084,5790,...,NaN,8600,0.101382,0.427426,0.091021,NaN,NaN,NaN,continuity,Awareness


In [13]:
# reorder columns
fb = fb[['date posted','link','message','episode','media type','phase','messaging','content series','asset type','subject','reach','organic impressions','shares','likes','comments','answers','vis eng','engagement','engagement rate','organic video views','organic video completes','view rate','completion rate']]
fb.head()

,date posted,link,message,episode,media type,phase,messaging,content series,asset type,subject,...,likes,comments,answers,vis eng,engagement,engagement rate,organic video views,organic video completes,view rate,completion rate
0,2019-03-23 09:25:00,https://www.facebook.com/kuwtk/posts/101560758...,"Some of you have never lost a $75,000 diamond ...",none,Status,continuity,Awareness,NaN,NaN,NaN,...,30761,5290,NaN,38823,105186,0.086503,0,0,0.000000,0.000000
1,2019-03-22 09:30:00,https://www.facebook.com/kuwtk/posts/101560723...,"These queens werk hard, but they play way hard...",none,Video,continuity,tune-in,NaN,NaN,NaN,...,6512,743,NaN,7858,228019,0.346015,432311,42212,0.482884,0.097643
2,2019-03-20 12:27:00,https://www.facebook.com/kuwtk/posts/101560700...,Mood,none,Photo,continuity,Awareness,NaN,NaN,NaN,...,20305,3913,NaN,27147,41842,0.045970,0,0,0.000000,0.000000
3,2019-03-17 09:00:00,https://www.facebook.com/kuwtk/posts/101560610...,Leave a ☘ to wish Bob a happy birthday!,none,Video,continuity,Awareness,NaN,NaN,NaN,...,3824,489,NaN,4427,45423,0.084823,284921,133726,0.399978,0.469344
4,2019-03-16 09:00:00,https://www.facebook.com/kuwtk/posts/101560610...,"11 years ago on this day, Kris wanted her very...",none,Video,continuity,Awareness,NaN,NaN,NaN,...,5790,2354,NaN,8600,99623,0.101382,561235,51084,0.427426,0.091021


In [14]:
fb.to_csv('fb_weekly.csv')

### IG Organic
Read in data export from Tracer, wrangle, and write to csv.

In [15]:
ig = pd.read_csv('tracer_ig_org_weekly_pull.csv')
print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
Page Name              15 non-null object
Post Created Date      15 non-null object
Post Permalink         14 non-null object
Post Media Type        14 non-null object
Post Caption           14 non-null object
Impressions            14 non-null float64
Reach                  14 non-null float64
Engagement             14 non-null float64
Post Like Count        14 non-null float64
Post Comments Count    14 non-null float64
Video Views            14 non-null float64
dtypes: float64(6), object(5)
memory usage: 1.5+ KB
None


,Page Name,Post Created Date,Post Permalink,Post Media Type,Post Caption,Impressions,Reach,Engagement,Post Like Count,Post Comments Count,Video Views
0,Kardashians on E!,3/7/19,https://www.instagram.com/p/ButwSd4Hk9E/,IMAGE,What @KrisJenner wants. The Momager gets 👌 #KUWTK,1265654.0,1048155.0,101780.0,91907.0,743.0,0.0
1,Kardashians on E!,3/8/19,https://www.instagram.com/p/BuwVU7JHaba/,VIDEO,"Shoutout to all the powerful moms, sisters, an...",622153.0,528646.0,20296.0,19455.0,41.0,322982.0
2,Kardashians on E!,3/8/19,https://www.instagram.com/p/BuwjppaHggF/,VIDEO,The Momager mastermind wins again 💯 #KUWTK,647916.0,566811.0,20013.0,18561.0,129.0,253864.0
3,Kardashians on E!,3/9/19,https://www.instagram.com/p/Buy6XYInc-G/,VIDEO,Which @KrisJenner are you vibing with this wee...,659192.0,538205.0,22354.0,18906.0,1097.0,217676.0
4,Kardashians on E!,3/11/19,https://www.instagram.com/p/Bu38GRvHHoY/,IMAGE,Double-tap to support this @KhloeKardashian ap...,1539462.0,1297922.0,111354.0,109671.0,429.0,0.0


In [16]:
#drop unwanted columns
ig.drop(columns = ['Page Name'], axis=1, inplace=True)
ig.drop(ig.index[-2:], inplace=True)
ig.head()

,Post Created Date,Post Permalink,Post Media Type,Post Caption,Impressions,Reach,Engagement,Post Like Count,Post Comments Count,Video Views
0,3/7/19,https://www.instagram.com/p/ButwSd4Hk9E/,IMAGE,What @KrisJenner wants. The Momager gets 👌 #KUWTK,1265654.0,1048155.0,101780.0,91907.0,743.0,0.0
1,3/8/19,https://www.instagram.com/p/BuwVU7JHaba/,VIDEO,"Shoutout to all the powerful moms, sisters, an...",622153.0,528646.0,20296.0,19455.0,41.0,322982.0
2,3/8/19,https://www.instagram.com/p/BuwjppaHggF/,VIDEO,The Momager mastermind wins again 💯 #KUWTK,647916.0,566811.0,20013.0,18561.0,129.0,253864.0
3,3/9/19,https://www.instagram.com/p/Buy6XYInc-G/,VIDEO,Which @KrisJenner are you vibing with this wee...,659192.0,538205.0,22354.0,18906.0,1097.0,217676.0
4,3/11/19,https://www.instagram.com/p/Bu38GRvHHoY/,IMAGE,Double-tap to support this @KhloeKardashian ap...,1539462.0,1297922.0,111354.0,109671.0,429.0,0.0


In [17]:
#rename columns
ig.columns = ['date','link','media type','caption','impressions','reach','engagement','likes','comments','video views']

#add columns
ig['episode'] = 'none'
ig['vis eng'] = ig['likes'] + ig['comments']
ig['eng rate'] = ig['engagement']/ig['impressions']
ig['view rate'] = ig['video views']/ig['impressions']

#Phase = premiere/continuity/finale
ig['phase'] = ig['date'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-03-25 00:00:00']) and x <= pd.to_datetime(['2019-04-04 00:00:00']) else "continuity")

#messaging column based on caption containing certain words - tune-in/awareness/catch-up
ig['messaging'] = ig['caption'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

ig['content series'] = np.nan

ig['asset type'] = np.nan

ig['subject'] = np.nan

print(ig.info())
ig

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 19 columns):
date              14 non-null object
link              14 non-null object
media type        14 non-null object
caption           14 non-null object
impressions       14 non-null float64
reach             14 non-null float64
engagement        14 non-null float64
likes             14 non-null float64
comments          14 non-null float64
video views       14 non-null float64
episode           14 non-null object
vis eng           14 non-null float64
eng rate          14 non-null float64
view rate         14 non-null float64
phase             14 non-null object
messaging         14 non-null object
content series    0 non-null float64
asset type        0 non-null float64
subject           0 non-null float64
dtypes: float64(12), object(7)
memory usage: 2.2+ KB
None


,date,link,media type,caption,impressions,reach,engagement,likes,comments,video views,episode,vis eng,eng rate,view rate,phase,messaging,content series,asset type,subject
0,3/7/19,https://www.instagram.com/p/ButwSd4Hk9E/,IMAGE,What @KrisJenner wants. The Momager gets 👌 #KUWTK,1265654.0,1048155.0,101780.0,91907.0,743.0,0.0,none,92650.0,0.080417,0.000000,continuity,Awareness,NaN,NaN,NaN
1,3/8/19,https://www.instagram.com/p/BuwVU7JHaba/,VIDEO,"Shoutout to all the powerful moms, sisters, an...",622153.0,528646.0,20296.0,19455.0,41.0,322982.0,none,19496.0,0.032622,0.519136,continuity,Awareness,NaN,NaN,NaN
2,3/8/19,https://www.instagram.com/p/BuwjppaHggF/,VIDEO,The Momager mastermind wins again 💯 #KUWTK,647916.0,566811.0,20013.0,18561.0,129.0,253864.0,none,18690.0,0.030888,0.391816,continuity,Awareness,NaN,NaN,NaN
3,3/9/19,https://www.instagram.com/p/Buy6XYInc-G/,VIDEO,Which @KrisJenner are you vibing with this wee...,659192.0,538205.0,22354.0,18906.0,1097.0,217676.0,none,20003.0,0.033911,0.330216,continuity,Awareness,NaN,NaN,NaN
4,3/11/19,https://www.instagram.com/p/Bu38GRvHHoY/,IMAGE,Double-tap to support this @KhloeKardashian ap...,1539462.0,1297922.0,111354.0,109671.0,429.0,0.0,none,110100.0,0.072333,0.000000,continuity,Awareness,NaN,NaN,NaN
5,3/12/19,https://www.instagram.com/p/Bu6kB8Sn-DS/,VIDEO,"October 14th, 2007 #KUWTK was born. #JunkInThe...",2008786.0,1541522.0,35089.0,27781.0,264.0,1174050.0,none,28045.0,0.017468,0.584457,continuity,Awareness,NaN,NaN,NaN
6,3/14/19,https://www.instagram.com/p/BvAd0UKH8rp/,VIDEO,"What do you prefer, fam: nude lip or red? Watc...",3450264.0,2584894.0,115282.0,98496.0,386.0,1811548.0,none,98882.0,0.033413,0.525046,continuity,Awareness,NaN,NaN,NaN
7,3/15/19,https://www.instagram.com/p/BvCQYp6HYMP/,VIDEO,"Beneath the glam, they’re just a fam 🤣 #KUWTK",7067761.0,5009781.0,246364.0,182843.0,1136.0,3636379.0,none,183979.0,0.034857,0.514502,continuity,Awareness,NaN,NaN,NaN
8,3/16/19,https://www.instagram.com/p/BvE10pKnFWh/,CAROUSEL_ALBUM,@KrisJenner wanted to eat organic before it wa...,2568133.0,1798392.0,84458.0,70752.0,775.0,0.0,none,71527.0,0.032887,0.000000,continuity,Awareness,NaN,NaN,NaN
9,3/17/19,https://www.instagram.com/p/BvHYu0EhVEX/,VIDEO,Leave a ☘ to wish Bob a happy birthday! #StPat...,2158314.0,1638382.0,58166.0,51852.0,542.0,1050697.0,none,52394.0,0.026950,0.486814,continuity,Awareness,NaN,NaN,NaN


In [18]:
#reorder columns
ig = ig[['date','link','caption','episode','media type','phase','messaging','content series','asset type','subject','reach','impressions','engagement','eng rate','likes','comments','vis eng','video views','view rate']]

print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 19 columns):
date              14 non-null object
link              14 non-null object
caption           14 non-null object
episode           14 non-null object
media type        14 non-null object
phase             14 non-null object
messaging         14 non-null object
content series    0 non-null float64
asset type        0 non-null float64
subject           0 non-null float64
reach             14 non-null float64
impressions       14 non-null float64
engagement        14 non-null float64
eng rate          14 non-null float64
likes             14 non-null float64
comments          14 non-null float64
vis eng           14 non-null float64
video views       14 non-null float64
view rate         14 non-null float64
dtypes: float64(12), object(7)
memory usage: 2.2+ KB
None


,date,link,caption,episode,media type,phase,messaging,content series,asset type,subject,reach,impressions,engagement,eng rate,likes,comments,vis eng,video views,view rate
0,3/7/19,https://www.instagram.com/p/ButwSd4Hk9E/,What @KrisJenner wants. The Momager gets 👌 #KUWTK,none,IMAGE,continuity,Awareness,NaN,NaN,NaN,1048155.0,1265654.0,101780.0,0.080417,91907.0,743.0,92650.0,0.0,0.000000
1,3/8/19,https://www.instagram.com/p/BuwVU7JHaba/,"Shoutout to all the powerful moms, sisters, an...",none,VIDEO,continuity,Awareness,NaN,NaN,NaN,528646.0,622153.0,20296.0,0.032622,19455.0,41.0,19496.0,322982.0,0.519136
2,3/8/19,https://www.instagram.com/p/BuwjppaHggF/,The Momager mastermind wins again 💯 #KUWTK,none,VIDEO,continuity,Awareness,NaN,NaN,NaN,566811.0,647916.0,20013.0,0.030888,18561.0,129.0,18690.0,253864.0,0.391816
3,3/9/19,https://www.instagram.com/p/Buy6XYInc-G/,Which @KrisJenner are you vibing with this wee...,none,VIDEO,continuity,Awareness,NaN,NaN,NaN,538205.0,659192.0,22354.0,0.033911,18906.0,1097.0,20003.0,217676.0,0.330216
4,3/11/19,https://www.instagram.com/p/Bu38GRvHHoY/,Double-tap to support this @KhloeKardashian ap...,none,IMAGE,continuity,Awareness,NaN,NaN,NaN,1297922.0,1539462.0,111354.0,0.072333,109671.0,429.0,110100.0,0.0,0.000000


In [19]:
#write to csv
ig.to_csv('tracer_ig_org_weekly.csv')